# Discrete Dynamical Systems with Julia!

Dynamical systems are rules describing the evolution of some variable $\vec{u}$ in time, according to some equations of motion given by a function $\vec{f}$.  
Here, we'll investigate _discrete dynamical systems_, often called _maps_, whereby time is discrete (in 'time-steps') and the equations of motion take the general form:

$$
\vec{u}_{n+1} = \vec{f}(\vec{u}_n,p,n)
$$

We'll start by using a performant julia library to achieve this, and then try to match that behavior ourselves using minimal julia code (adapted from Chris Rackauckas's [Parallel Computing and Scientific Machine Learning](https://github.com/mitmath/18337) course notes).

## DynamicalSystems.jl

[DynamicalSystems.jl](https://juliadynamics.github.io/DynamicalSystems.jl/latest/) is a performant Julia software library for dynamical systems, deterministic chaos and nonlinear timeseries analysis.  

We start by importing the necessary packages

In [ ]:
using DynamicalSystems, Plots, StaticArrays, BenchmarkTools

### De Jong attractor
Let's have a look at the following attractor:
$$
\begin{align}
x_{n+1} &= sin(a y_n) - cos(b x_n) \\
y_{n+1} &= sin(c x_n) - cos(d y_n)
\end{align}
$$

we define a function to perform this map:

In [ ]:
function dejong_eom(x,p,t)
    a,b,c,d = p
    dx = sin(a*x[2]) - cos(b*x[1])
    dy = sin(c*x[1]) - cos(d*x[2])
    @SVector [dx, dy]
end

Next, we need an initial state to start our iteration:

In [ ]:
initial_state = @SVector([1.0 ,1.0]);

and finally, our parameters, $p$:

In [ ]:
p = [-2.0,-2.0,-1.2,2.0];

now, we can define our `DynamicalSystem`:

In [ ]:
dejong = DiscreteDynamicalSystem(dejong_eom,initial_state,p)

obtaining a trajectory for some 100000 points is then straightforward:

In [ ]:
trajectory_dejong = trajectory(dejong,100000)

We can plot the trajectory to visualize our attractor:

In [ ]:
scatter(trajectory_dejong[:,1], trajectory_dejong[:,2], 
    markersize=0.1, markeralpha = 0.3, markercolor="black",
    leg=false, title="De Jong attractor", 
    html_output_format=:png, size=(1200,800))

#### Performance
Let's evaluate the performance of the above code. We'll use the `BenchmarkTools` package:

In [ ]:
@btime trajectory(dejong,10000);

This is indeed quite fast, let's see if we can match this using our own implementation.

## Minimal Implementation of Discrete Dynamical Systems
We'll follow the same paradigm as `DynamicalSystems.jl` to code this efficiently. Namely, we'll define a _higher order function_ which will accept our equations of motion as an argument and type-specialize on that function during compilation

In [ ]:
# Higher order function
function solve_discrete_map(f,u0,p,n)
    # Set initial state
    u = u0
    for i in 1:n-1
        # Iteratively apply map n times
        u = f(u,p)
    end
    # Return final result
    u
end

In [ ]:
function dejong_eom_naive(u,p)
    a,b,c,d = p
    du1 = sin(a*u[2]) - cos(b*u[1])
    du2 = sin(c*u[1]) - cos(d*u[2])
    [du1, du2]
end

initial_state = [1.0, 1.0];
p = (-2.0,-2.0,-1.2,2.0);

@btime solve_discrete_map(dejong_eom_naive, initial_state, p , 10000)

This is already quite fast!  
But, we're making a lot of costly allocations. Also, note this only returns the final point - not the whole trajectory.  
Let's use statically-sized arrays to save the entire trajectory:

In [ ]:
function solve_discrete_map_save(f,u0,p,n)
    # Set statically-sized output array
    u = Vector{typeof(u0)}(undef,n)
    u[1] = u0
    for i in 1:n-1
        u[i+1] = f(u[i],p)
    end
    u
end

@btime solve_discrete_map_save(dejong_eom_naive, initial_state, p , 10000)

We now obtain the full trajectory! It seems natural to use matrices instead, since our data is a full 2D array.

In [ ]:
function solve_discrete_map_save_matrix(f,u0,p,n)
    # Set statically-sized output array
    u = Matrix{eltype(u0)}(undef,length(u0),n)
    u[:,1] = u0
    for i in 1:n-1
        u[:,i+1] = f(u[:,i],p)
    end
    u
end

@btime solve_discrete_map_save_matrix(dejong_eom_naive, initial_state, p , 10000)

What went wrong? this is almost twice as slow with twice as many allocations!  
The problem is that slicing automatically makes a copy of the array in julia. To remedy this, we can use the `@view` macro.

In [ ]:
function solve_discrete_map_save_matrix(f,u0,p,n)
    # Set statically-sized output array
    u = Matrix{eltype(u0)}(undef,length(u0),n)
    u[:,1] = u0
    for i in 1:n-1
        u[:,i+1] = f(@view(u[:,i]),p)
    end
    u
end

@btime solve_discrete_map_save_matrix(dejong_eom_naive, initial_state, p , 10000)

Notice we used `StaticArrays` in the beginning with `DynamicalSystems`. Indeed, we can leverage the same functionality since our map is only two-dimensional:

In [ ]:
function dejong_eom_static(u,p)
    a,b,c,d = p
    du1 = sin(a*u[2]) - cos(b*u[1])
    du2 = sin(c*u[1]) - cos(d*u[2])
    @SVector [du1, du2]
end

@btime solve_discrete_map_save(dejong_eom_static,@SVector[1.0,1.0], p , 10000);

This is in-fact faster than the `DynamicalSystems` evaluation!  
Going further, since we know we're always accessing a 2D array, we can turn off bounds checks.

In [ ]:
function dejong_eom_fast(u,p)
    a,b,c,d = p
    @inbounds begin
    du1 = sin(a*u[2]) - cos(b*u[1])
    du2 = sin(c*u[1]) - cos(d*u[2])
    end
    @SVector [du1, du2]
end

function solve_discrete_map_fast(f,u0,p,n)
    u = Vector{typeof(u0)}(undef,n)
    @inbounds u[1] = u0
    @inbounds for i in 1:n-1
        u[i+1] = f(u[i],p)
    end
    u
end

@btime solve_discrete_map_fast(dejong_eom_fast,@SVector[1.0,1.0], p , 10000);

Finally, we can eliminate all allocations by using an in-place function:

In [ ]:
function solve_discrete_map_fast!(u,f,u0,p,n)
    @inbounds u[1] = u0
    @inbounds for i in 1:length(u)-1
        u[i+1] = f(u[i],p)
    end
    u
end

u = Vector{typeof(@SVector[1.0,1.0])}(undef,10000)
@btime solve_discrete_map_fast!(u,dejong_eom_fast,@SVector[1.0,1.0], p , 10000);

### Binning along the way
Notice we've made the iteration loop very fast, but plotting all these points will still be slow. Instead, we can count the number of hits, binning along the way.  
First, let's write a mutating version of our attractor:

In [ ]:
function dejong_eom_fast_ip(du,u,p)
    a,b,c,d = p
    @inbounds begin
    du[1] = sin(a*u[2]) - cos(b*u[1])
    du[2] = sin(c*u[1]) - cos(d*u[2])
    end
end

function solve_discrete_map_mutate(f,u0,p,n)
    du = similar(u0)
    u  = copy(u0)
    for i in 1:n-1
        f(du,u,p)
        u,du = du,u
    end
    u
end

@btime solve_discrete_map_mutate(dejong_eom_fast_ip,[1.0,1.0], p , 10000)

Finally, let's add in-place binning logic:

In [ ]:
function solve_discrete_map_mutate!(bins,f,u0,p,n,op)
    
    xmin,xmax,ymin,ymax,delta = op
    dimx, dimy = size(bins)
    
    du = similar(u0)
    u  = copy(u0)
    
    @inbounds for i in 1:n-1
        f(du,u,p)
        u,du = du,u
        
        tx = floor(Int,(u[1]-xmin)/delta) +1
        ty = floor(Int,(u[2]-ymin)/delta) +1
        if tx >= 1 && tx <= dimx && ty >= 1 && ty <= dimy
            bins[tx,ty] +=1
        end
        
    end
end

In [ ]:
xmin,xmax,ymin,ymax,delta = (-2.5,2.5,-2.5,2.5,0.005)
dimx = floor(Int,(xmax -xmin)/delta) +1
dimy = floor(Int,(ymax -ymin)/delta) +1
bins = zeros(Int,dimx,dimy);

@btime solve_discrete_map_mutate!(bins,dejong_eom_fast_ip,[1.0,1.0], p , 10000,(xmin,xmax,ymin,ymax,delta))

In [ ]:
bins = zeros(Int,dimx,dimy);
@btime solve_discrete_map_mutate!(bins,dejong_eom_fast_ip,[1.0,1.0], p , 10000000,(xmin,xmax,ymin,ymax,delta))

In [ ]:
heatmap(log.(bins.+1))